## Fazendo Importações e passando os parâmetros necessários para realizar as resquisições

In [ ]:
import hashlib
import time
import requests
from google.colab import userdata
import sqlite3
import pandas as pd

public_key = userdata.get('public_key')
private_key = userdata.get('private_key')

ts = str(time.time())
to_hash = ts + private_key + public_key
hash_digest = hashlib.md5(to_hash.encode()).hexdigest()
auth_params = {
        "ts": ts,
        "apikey": public_key,
        "hash": hash_digest,
        "limit": 100
    }

In [ ]:
url1 = "https://gateway.marvel.com/v1/public/characters"
url2 = "https://gateway.marvel.com/v1/public/comics"

## Função que consome o endpoint da API

In [ ]:
def get_data(url):
  total_data = 0
  offset = 0
  data = pd.DataFrame()

  # Garante a entrada no loop na primeira vez
  while offset < total_data or total_data == 0:
      print(offset)
      auth_params['offset'] = offset
      response = requests.get(url, auth_params)
      response = response.json()

      # Atualiza total_data após a primeira requisição
      if total_data == 0:
          total_data = response['data']['total']

      # Verifica se a resposta contém dados
      if not response['data']['results']:
          break

      response = pd.json_normalize(response['data']['results'])
      df1 = response
      data = pd.concat([data, df1])
      offset += len(df1)

  return data

In [ ]:
characters = get_data(url1)

conn = sqlite3.connect('banco.db')
columns = ['id', 'name','description']

characters[columns].to_sql('characters', conn, if_exists='replace', index=False)

conn.close()

In [ ]:
conn = sqlite3.connect('banco.db')

query = "SELECT id FROM characters"
df = pd.read_sql_query(query, conn)

conn.close()

In [ ]:
offset = 0
total = None
request_count = 0
comics = []

print("Buscando todos os quadrinhos...")

for index, row in df.iterrows():
  char_id = row['id']
  print(char_id)
  if index == 3:
    break

  while offset >= total::
      print(offset)
      comic_params = params
      comic_params.update({
          'offset': offset
      })

      comic_resp = requests.get(f"{BASE_URL}/characters/{char_id}/comics", params=comic_params)
      request_count += 1

      comics_data = comic_resp.json()
      results = comics_data['data']['results']
      comics.extend(results)

      if total is None:
          total = comics_data['data']['total']
          print(f"Total de quadrinhos encontrados: {total}")

      offset += limit

      # if offset >= total:
      #     break

In [ ]:
columns = ['id', 'name','description']
characters[columns].to_sql('characters', conn, if_exists='replace', index=False)

In [ ]:
request_count = 0
comics_df = pd.DataFrame()
limit = 100

for index, row in df.iterrows():
  char_id = row['id']
  print(index)
  offset = 0
  total = 0

  # Loop enquanto o offset for menor que o total ou total ainda não foi definido
  while offset <= total or total == 0:
      comic_params = params
      comic_params.update({
          'offset': offset
      })

      comic_resp = requests.get(f"{BASE_URL}/characters/{char_id}/comics", params=comic_params)
      request_count += 1

      comics_data = comic_resp.json()
      results = comics_data['data']['results']

      temp_df = pd.DataFrame(results)

      temp_df['character_id'] = char_id

      comics_df = pd.concat([comics_df, temp_df], ignore_index=True)

      if total == 0:
          total = comics_data['data']['total']
          if total == 0:
            break

      if total < limit:
        offset += total
      else:
        offset += limit